In [1]:
from datasets import Dataset
from PIL import Image
import json, os

In [3]:
os.getcwd()

'/content'

In [2]:
image_root = "/Users/yashwanth/Documents/OMSCS/Deep_Learning/dl_project_fall_2025/data/random_images"
captions_path = os.path.join(image_root, "captions.json")

with open(captions_path) as f:
    raw_samples = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yashwanth/Documents/OMSCS/Deep_Learning/dl_project_fall_2025/data/random_images/captions.json'

In [4]:
system_message = (
    "You are Yash, a precise but informal expert describing images. "
    "Use my personal style: compact, concrete, slightly conversational, no extra fluff."
)


def build_sample(row):
    img_path = os.path.join(image_root, "images", row["file"])
    image = Image.open(img_path).convert("RGB")

    # Chat-style structure for Qwen2-VL
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text",  "text": "Describe this image in my style."},
            ],
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": row["caption"]},
            ],
        },
    ]
    return {
        "images": [image],   # TRL uses this list
        "messages": messages
    }

In [5]:
formatted = [build_sample(r) for r in raw_samples]

NameError: name 'raw_samples' is not defined

In [28]:
full_ds = Dataset.from_list(formatted)
# tiny split: 12 train / 3 eval
train_dataset = full_ds.select(range(7))
eval_dataset  = full_ds.select(range(7, len(full_ds)))

In [30]:
# import torch
# from transformers import (
#     Qwen2VLForConditionalGeneration,
#     Qwen2VLProcessor,
#     BitsAndBytesConfig,
# )

# model_id = "Qwen/Qwen2-VL-2B-Instruct"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     quantization_config=bnb_config,
# )

# processor = Qwen2VLProcessor.from_pretrained(model_id)

import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor

model_id = "Qwen/Qwen2-VL-2B-Instruct"

device = "mps" if torch.backends.mps.is_available() else "cpu"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "mps" else torch.float32,
)
model.to(device)

processor = Qwen2VLProcessor.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.65s/it]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


In [37]:
# If TRL is missing, uncomment the next line to install it in this env
# %pip install -q trl

from trl import SFTTrainer
from transformers import TrainingArguments

# Ensure pad token is set for loss masking
if processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id

def add_prompt(example):
    example["prompt"] = processor.apply_chat_template(
        example["messages"], tokenize=False, add_generation_prompt=False
    )
    example["image"] = example["images"][0]
    return example

train_for_trl = train_dataset.map(add_prompt)
eval_for_trl = eval_dataset.map(add_prompt)

def data_collator(batch):
    prompts = [row["prompt"] for row in batch]
    images = [row["image"] for row in batch]
    model_inputs = processor(
        text=prompts,
        images=images,
        padding=True,
        return_tensors="pt",
    )
    labels = model_inputs["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

training_args = TrainingArguments(
    output_dir="./outputs/qwen2vl_sft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_steps=1,
    save_steps=50,
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=0,
)

sft_trainer = SFTTrainer(
    model=model,
    tokenizer=processor.tokenizer,
    train_dataset=train_for_trl,
    eval_dataset=eval_for_trl,
    args=training_args,
    data_collator=data_collator,
    dataset_text_field="prompt",
    max_seq_length=None,
)

sft_trainer.train()


Map: 100%|██████████| 3/3 [00:09<00:00,  3.13s/ examples]


TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'